<a href="https://colab.research.google.com/github/DaveAlmR/CadastroCliente/blob/master/ResNet50_Melanoma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import scipy.io
from sklearn import model_selection
import tensorflow as tf


#trash
import matplotlib
import gc
#matplotlib.pyplot.imshow(ML1.mean(axis=2))

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import os
import pandas as pd

In [2]:
!nvidia-smi -L
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import sys
sys.path.append('/content/drive/My Drive/colab/cortes')
from Melanoma import Melanoma
from NevoDisplasico import NevoDisplasico
from PeleSaudavel import PeleSaudavel

In [5]:
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [6]:
def padImage(image, x=2094, y=893, z=230):
  aux = np.zeros((x,y,z), dtype="float32")
  aux[0:image.shape[0],0:image.shape[1],0:image.shape[2]] = image

  del image
  gc.collect()
  
  return aux

In [7]:
def GetContent(path, objectClass, len=0):
  DATA = scipy.io.loadmat(path)
  samples = objectClass.get_samples()
  array = []
  i=0

  for sample in samples:
    i+=1
    if (i > len and len != 0):
      break

    parts = objectClass.get_parts_of_sample(sample)
    data = DATA[sample].astype("float32")

    for part in range(parts):
      x1, x2, y1, y2 = objectClass.get_sample_ref(sample, part+1)
      array.append(data[y1:y2, x1:x2])
  
  del DATA
  gc.collect()
  return array


In [8]:
def GetMax(array):
  x = 0
  y = 0

  for value in array:
    x = max(x, value.shape[0])
    y = max(y, value.shape[1])
    
  return x, y


In [9]:
def reduceDimensionality(HSI, min_explained_variance_ratio=0.99, PCs=None):
	X = HSI.reshape((HSI.shape[0] * HSI.shape[1], HSI.shape[2]))
	
	if PCs == None:
		pca = PCA(min_explained_variance_ratio)
	else:
		pca = PCA(n_components=PCs)
  
	pca.fit(X)

	data_reduced = pca.transform(X)
	data_reduced = data_reduced.reshape(HSI.shape[0], HSI.shape[1], -1)
	
	return data_reduced
	
def reduceDimensionalityOfArray(hsiArray, min_explained_variance_ratio=0.99, PCs=None):
	for i in range(len(hsiArray)):
		hsiArray[i] = reduceDimensionality(hsiArray[i], min_explained_variance_ratio, PCs)

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, GlobalAveragePooling2D, Dropout
from keras.models import Model

def GetModel(input_shape):
		model = tf.keras.applications.resnet50.ResNet50(input_shape=input_shape, include_top=False,weights=None, classes=3)
		x = model.output
		x = GlobalAveragePooling2D()(x)
		x = Dense(512, activation='relu')(x)
		x = Dropout(0.5)(x)
		x = Dense(512, activation='relu')(x)
		x = Dropout(0.5)(x)
		out = Dense(3,activation='softmax')(x)
		return Model(inputs=model.input,outputs=out)
  
    

In [11]:
#pca = PCA()

#melanoma = GetContent('/content/drive/MyDrive/colab/Melanoma/ML_Original.mat', Melanoma(), 12)
#rep_melanoma = [sum(pca.explained_variance_ratio_[0:4]) for pca in [pca.fit(x) for x in [HSI.reshape((HSI.shape[0] * HSI.shape[1], HSI.shape[2])) for HSI in melanoma]]]
#del melanoma
#gc.collect()

#nevoDisplasico = GetContent('/content/drive/MyDrive/colab/Melanoma/ND_Original.mat', NevoDisplasico(), 18)
#rep_nevoDisplasico = [sum(pca.explained_variance_ratio_[0:4]) for pca in [pca.fit(x) for x in [HSI.reshape((HSI.shape[0] * HSI.shape[1], HSI.shape[2])) for HSI in nevoDisplasico]]]
#del nevoDisplasico
#gc.collect()

#peleSaudavel = GetContent('/content/drive/MyDrive/colab/Melanoma/PS_Original.mat', PeleSaudavel(), 5)
#rep_peleSaudavel = [sum(pca.explained_variance_ratio_[0:4]) for pca in [pca.fit(x) for x in [HSI.reshape((HSI.shape[0] * HSI.shape[1], HSI.shape[2])) for HSI in peleSaudavel]]]
#del peleSaudavel
#gc.collect()

#print(rep_melanoma)
#print(rep_nevoDisplasico)
#print(rep_peleSaudavel)

In [12]:
if False:
  PCs = 3

  melanoma = GetContent('/content/drive/MyDrive/colab/Melanoma/ML_Original.mat', Melanoma(), 12)
  reduceDimensionalityOfArray(melanoma, PCs=PCs)

  nevoDisplasico = GetContent('/content/drive/MyDrive/colab/Melanoma/ND_Original.mat', NevoDisplasico(), 18)
  reduceDimensionalityOfArray(nevoDisplasico, PCs=PCs)

  peleSaudavel = GetContent('/content/drive/MyDrive/colab/Melanoma/PS_Original.mat', PeleSaudavel(), 5)
  reduceDimensionalityOfArray(peleSaudavel, PCs=PCs)

  NMelanoma = min(12, len(melanoma))
  NNevoDisplasico = min(72, len(nevoDisplasico))
  NPeleSaudavel = min(17, len(peleSaudavel))

  melanoma = melanoma[0:NMelanoma]
  nevoDisplasico = nevoDisplasico[0:NNevoDisplasico]
  peleSaudavel = peleSaudavel[0:NPeleSaudavel]

  data = melanoma + nevoDisplasico + peleSaudavel
  data = np.array(data)

  del melanoma
  del nevoDisplasico
  del peleSaudavel
  gc.collect()

  maxX, maxY = GetMax(data)
  for i in range(len(data)):
    data[i] = padImage(data[i], maxX, maxY, PCs)

  x_data = np.stack(data)
  y_data = NMelanoma*[0] + NNevoDisplasico*[1] + NPeleSaudavel*[2]

  np.save('/content/drive/MyDrive/colab/Melanoma/x_data',x_data)
  np.save('/content/drive/MyDrive/colab/Melanoma/y_data',y_data)

  np.save('/content/drive/MyDrive/colab/Melanoma/NMelanoma',NMelanoma)
  np.save('/content/drive/MyDrive/colab/Melanoma/NNevoDisplasico',NNevoDisplasico)
  np.save('/content/drive/MyDrive/colab/Melanoma/NPeleSaudavel',NPeleSaudavel)
else:
  x_data = np.load('/content/drive/MyDrive/colab/Melanoma/x_data.npy')
  y_data = np.load('/content/drive/MyDrive/colab/Melanoma/y_data.npy')

  NMelanoma = np.load('/content/drive/MyDrive/colab/Melanoma/NMelanoma.npy')
  NNevoDisplasico = np.load('/content/drive/MyDrive/colab/Melanoma/NNevoDisplasico.npy')
  NPeleSaudavel = np.load('/content/drive/MyDrive/colab/Melanoma/NPeleSaudavel.npy')

  maxX = x_data.shape[1]
  maxY = x_data.shape[2]
  PCs = x_data.shape[3]


In [13]:
from random import shuffle

num_classes = 3
batch_size = 2
input_shape = (maxX, maxY, PCs)
test_size = 0.4
val_size = 0.5

y_data = keras.utils.to_categorical(y_data, num_classes)

x_melanoma = x_data[0:NMelanoma]
y_melanoma = y_data[0:NMelanoma]

x_nevoDisplasico = x_data[NMelanoma:NMelanoma+NNevoDisplasico]
y_nevoDisplasico = y_data[NMelanoma:NMelanoma+NNevoDisplasico]

x_peleSaudavel = x_data[NMelanoma+NNevoDisplasico:]
y_peleSaudavel = y_data[NMelanoma+NNevoDisplasico:]

x_train, x_test, y_train, y_test = model_selection.train_test_split(x_melanoma, y_melanoma, test_size=test_size)

x_test, x_val, y_test, y_val = model_selection.train_test_split(x_test, y_test, test_size=val_size)

x_train_aux, x_test_aux, y_train_aux, y_test_aux = model_selection.train_test_split(x_nevoDisplasico, y_nevoDisplasico, test_size=test_size)
x_train = np.concatenate((x_train, x_train_aux))
y_train = np.concatenate((y_train, y_train_aux)) 

x_test_aux, x_val_aux, y_test_aux, y_val_aux = model_selection.train_test_split(x_test_aux, y_test_aux, test_size=val_size)
x_test = np.concatenate((x_test, x_test_aux)) 
y_test = np.concatenate((y_test, y_test_aux)) 
x_val = np.concatenate((x_val, x_val_aux)) 
y_val = np.concatenate((y_val, y_val_aux)) 

x_train_aux, x_test_aux, y_train_aux, y_test_aux = model_selection.train_test_split(x_peleSaudavel, y_peleSaudavel, test_size=test_size)
x_train = np.concatenate((x_train, x_train_aux))
y_train = np.concatenate((y_train, y_train_aux)) 

x_test_aux, x_val_aux, y_test_aux, y_val_aux = model_selection.train_test_split(x_test_aux, y_test_aux, test_size=val_size)
x_test = np.concatenate((x_test, x_test_aux)) 
y_test = np.concatenate((y_test, y_test_aux)) 
x_val = np.concatenate((x_val, x_val_aux)) 
y_val = np.concatenate((y_val, y_val_aux)) 

del x_data
del y_data
del x_test_aux
del y_test_aux
del x_val_aux
del y_val_aux
del x_train_aux
del y_train_aux
gc.collect()

ind_list = [i for i in range(y_train.shape[0])]
shuffle(ind_list)

x_train = x_train[ind_list]
y_train = y_train[ind_list]


ind_list = [i for i in range(y_test.shape[0])]
shuffle(ind_list)

x_test = x_test[ind_list]
y_test = y_test[ind_list]


ind_list = [i for i in range(y_val.shape[0])]
shuffle(ind_list)

x_val = x_val[ind_list]
y_val = y_val[ind_list]

In [14]:
def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer._decayed_lr(tf.float32) # I use ._decayed_lr method instead of .lr
    return lr

optimizer = keras.optimizers.Adam()
lr_metric = get_lr_metric(optimizer)

In [15]:
#use_tpu = True #@param {type:"boolean"}
use_tpu = 'COLAB_TPU_ADDR' in os.environ
use_gpu = '__EGL_VENDOR_LIBRARY_DIRS' in os.environ

print(use_tpu)
print(use_gpu)

if use_tpu:
  assert 'COLAB_TPU_ADDR' in os.environ, 'Missing TPU; did you request a TPU in Notebook Settings?'

  if 'COLAB_TPU_ADDR' in os.environ:
    TF_MASTER = 'grpc://{}'.format(os.environ['COLAB_TPU_ADDR'])
  else:
    TF_MASTER=''

  tpu_address = TF_MASTER

  resolver = tf.distribute.cluster_resolver.TPUClusterResolver(TF_MASTER)
  tf.config.experimental_connect_to_cluster(resolver)
  tf.tpu.experimental.initialize_tpu_system(resolver)
  strategy = tf.distribute.experimental.TPUStrategy(resolver)
  
  with strategy.scope():
    model = GetModel(input_shape)
    model.compile(loss=keras.losses.mse, optimizer=keras.optimizers.Adam(learning_rate=0.1), metrics=["accuracy", lr_metric])

elif use_gpu:
  with tf.device('/device:GPU:0'):
    model = model = GetModel(input_shape)
    model.compile(loss="categorical_crossentropy", optimizer="Adam", metrics=["accuracy"])
else:
    print("this")
    model = GetModel(input_shape)
    model.compile(loss=keras.losses.mse, optimizer=keras.optimizers.Adam(learning_rate=0.1), metrics=["accuracy", lr_metric])

model.summary()

False
False
this
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 820, 1594, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 826, 1600, 3) 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 410, 797, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 410, 797, 64) 256         conv1_conv[0][0]                 
_____________________________________________________________________________

In [16]:
class MyCustomCallback(tf.keras.callbacks.Callback):
  def on_train_begin(self, logs=None):
    gc.collect()
  def on_epoch_end(self, epoch, logs=None):
    gc.collect()
  def on_train_batch_end(self, batch, logs=None):
    gc.collect()

In [ ]:
epochs = 200
es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)
#tf.config.run_functions_eagerly(True)

history = model.fit(x_train[0:56]
          ,y_train[0:56]
          ,batch_size=8
          ,epochs=epochs
          ,validation_data=(x_val, y_val)
          ,callbacks=[es_callback])

Epoch 1/200


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, ConfusionMatrixDisplay

score = model.evaluate(x_test, y_test, verbose=0)

print("Test loss:", score[0])
print("Test accuracy:", score[1])

plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['accuracy', 'val_accuracy'], loc='upper left')
plt.show()

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['loss', 'val_loss'], loc='upper left')
plt.show()

predicted_classes = model.predict(x_test)
predicted_classes = np.argmax(np.round(predicted_classes), axis=1)
y_test = np.argmax(np.round(y_test), axis=1)
target_names = ["Class {}".format(i) for i in range(3)]
print(classification_report(y_test, predicted_classes, target_names=target_names))

In [ ]:
#model.save_weights('/content/drive/MyDrive/colab/Melanoma/ResNet50.h5', overwrite=True)
path = '/content/drive/MyDrive/colab/Melanoma/ResNet50'
tf.keras.models.save_model(model, path)
#loaded_model = tf.keras.models.load_model(path)
